In [3]:
import pandas as pd 
import json 
import utils 




In [4]:
file_path = "./outputs/demo_google_claude_3.5_sonnet_v2_2024-12-03_0.json"
data = pd.DataFrame(utils.load_json(file_path))

In [6]:
for i, row in data.iterrows():
    print(f'The problem {row['name']} test results : {row['pass']}')
    # print(f'The problem {row['name']} test results : {row['stderr']}')
    print('-' * 20)
    

The problem test_eth_parser_lite_2 test results : True
--------------------
The problem test_l2_switch_lite_5 test results : False
--------------------
The problem test_axist_storefwd_fifo_4 test results : False
--------------------
The problem test_axist_rate_limit_3 test results : False
--------------------
The problem test_eth_parser_4 test results : False
--------------------
The problem test_axist_storefwd_fifo_3 test results : False
--------------------
The problem test_axist_rate_limit_2 test results : False
--------------------
The problem test_l2_switch_6 test results : False
--------------------
The problem test_eth_parser_2 test results : False
--------------------
The problem test_axist_rate_limit_lite_3 test results : False
--------------------
The problem test_axist_rrarb_3 test results : False
--------------------
The problem test_axist_simple_fifo_3 test results : False
--------------------
The problem test_axist_storefwd_fifo_5 test results : False
--------------------

In [29]:
name = 'test_eth_parser_lite_2'
steps = data[data['name'] == name]['conversation'][0].split('\n [NEW STEP]\n')
# for step in steps:
#     print(step)
#     print('-' *30)
print(data[data['name'] == name]['dut'][0])

module dut #(
    parameter type AxiStreamType = local_pkg::axi4st_8_s,
    parameter int unsigned AxiStreamWidth = 8
) (
    input  logic        clock,
    input  logic        reset,
    input  AxiStreamType inAxi4St,
    output logic        inTready,
    output AxiStreamType outAxi4St,
    input  logic        outTready,
    output logic        outParsedValid,
    output logic [47:0] outParsedDestMac,
    output logic [47:0] outParsedSourceMac
);

    // Internal signals
    logic frame_start;
    logic frame_end;
    logic data_valid;
    logic [3:0] byte_count;
    logic capture_enable;
    logic frame_active;
    logic parse_valid;
    logic [47:0] parsed_dest_mac;
    logic [47:0] parsed_source_mac;
    logic [AxiStreamWidth-1:0] data_out;

    // Frame state tracking
    typedef enum logic [1:0] {
        IDLE,
        CAPTURE_MAC,
        FORWARD
    } state_t;
    
    state_t current_state, next_state;
    
    // State register
    always_ff @(posedge clock) begin
        if 

In [11]:
data.head()

,return_code,stdout,stderr,pass,score,warnings,errors,runtime,name,index,...,dut_name,tb_name,sim_tool,pass_string,compile_only,timeout,dir,start_time,stop_time,conversation
0,0,,%Error: dut.sv:41:5: Cannot find file containi...,False,20,0,6,0.260206,test_eth_parser_lite_2,0,...,dut,tb,verilator,TEST PASS,False,60,work/job_68f1aad5-9471-4e26-a268-ca34876835cd_1,1.733169e+09,1.733169e+09,Problem description:\n ======\n Create a...
1,0,,%Error: dut.sv:33:13: Cannot find file contain...,False,20,0,6,0.321635,test_l2_switch_lite_5,1,...,dut,tb,verilator,TEST PASS,False,60,work/job_68f1aad5-9471-4e26-a268-ca34876835cd_2,1.733169e+09,1.733169e+09,Problem description:\n ======\n Create a...
2,0,make: Entering directory '/home/ubuntu/rtl-puz...,%Warning-PINMISSING: dut.sv:101:11: Cell has m...,False,79,1,0,10.438988,test_axist_storefwd_fifo_4,2,...,dut,tb,verilator,TEST PASS,False,60,work/job_68f1aad5-9471-4e26-a268-ca34876835cd_0,1.733169e+09,1.733169e+09,Problem description:\n ======\n Create a...
3,0,make: Entering directory '/home/ubuntu/rtl-puz...,%Warning-PINMISSING: dut.sv:84:7: Cell has mis...,False,62,18,0,0.957248,test_axist_rate_limit_3,3,...,dut,tb,verilator,TEST PASS,False,60,work/job_68f1aad5-9471-4e26-a268-ca34876835cd_3,1.733169e+09,1.733169e+09,Problem description:\n ======\n Create a...
4,0,,%Error-UNSUPPORTED: dut.sv:154:5: Unsupported:...,False,40,0,4,0.317100,test_eth_parser_4,4,...,dut,tb,verilator,TEST PASS,False,60,work/job_68f1aad5-9471-4e26-a268-ca34876835cd_4,1.733169e+09,1.733169e+09,Problem description:\n ======\n Create a...


In [7]:
()

()